In [6]:
import MySQLdb
import MySQLdb.cursors
import json
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from tqdm import trange
from collections import defaultdict

class PopularNews():
    def __init__(self) -> None:
        db_config = json.load(open("../../db_config.json"))
        self._db = MySQLdb.connect(host = db_config["HOST"], user = db_config["USER"], passwd = db_config["PASSWD"],
                                    db = "financial", charset = "utf8", cursorclass = MySQLdb.cursors.DictCursor)
        self._cursor = self._db.cursor()

    def _init_table(self) -> None:
        query = "TRUNCATE TABLE popular_news"
        self._cursor.execute(query)
        self._db.commit()

    def _insert(self, ticker_id : str, news_id : str, interval : str) -> None:
        query = "INSERT INTO popular_news (ticker_id, news_id, time_interval) VALUES (%s, %s, %s)"
        param = (ticker_id, news_id, interval)

        self._cursor.execute(query, param)
        self._db.commit()

    def _get_news(self, interval : str = "3days") -> pd.DataFrame:
        if interval == "3days":
            start_date = (datetime.now() - timedelta(days = 3)).strftime("%Y-%m-%d")

        elif interval == "week":
            start_date = (datetime.now() - timedelta(days = 7)).strftime("%Y-%m-%d")

        elif interval == "month":
            start_date = (datetime.now() - relativedelta(months = 1)).strftime("%Y-%m-%d")

        query = "SELECT * FROM news WHERE date>=%s AND date<=%s"
        param = ("2023-02-01", "2023-02-07")

        self._cursor.execute(query, param)
        self._db.commit()

        result = pd.DataFrame.from_dict(self._cursor.fetchall())
        
        return result

    def _get_ticker_list(self) -> pd.DataFrame:
        query = "SELECT ID, stock_name FROM ticker_list WHERE LENGTH(stock_num)=4"

        self._cursor.execute(query,)
        self._db.commit()

        result = pd.DataFrame.from_dict(self._cursor.fetchall())

        stock_name = result["stock_name"].to_list()
        stock_name = [element[5:] for element in stock_name]

        result["stock_name"] = stock_name

        result = result.sort_values(by = ["stock_name"], ascending = False, key = lambda x: x.str.len())

        return result
    
    def _get_ticker_id(self, stock_num : str):
        query = "SELECT ID FROM ticker_list WHERE stock_num=%s"
        param = (stock_num,)

        self._cursor.execute(query, param)
        self._db.commit()

        result = pd.DataFrame.from_dict(self._cursor.fetchall())
        
        return result["ID"][0]

    def _handle_news(self, interval : str) -> None:
        result = defaultdict(list)

        news = self._get_news(interval = interval)
        ticker_list = self._get_ticker_list()

        for index_news in trange(len(news)):
            for index_ticker_list in range(len(ticker_list)):
                if ticker_list.iloc[index_ticker_list]["stock_name"] in news.iloc[index_news]["title"]:
                    if ticker_list.iloc[index_ticker_list]["stock_name"] == "冠軍":
                        if "隱形冠軍" in news.iloc[index_news]["title"]:
                            continue

                    result[ticker_list.iloc[index_ticker_list]["ID"]].append([news.iloc[index_news]["ID"], news.iloc[index_news]["title"], news.iloc[index_news]["link"]])
                    break
        
        result = {key : value for key, value in sorted(result.items(), key = lambda item : len(item[1]), reverse = True)}

        return result

    def run(self):
        # self._init_table()
        return self._handle_news(interval = "3days")
        # self._handle_news(interval = "week")
        # self._handle_news(interval = "month")

In [7]:
popular_news = PopularNews()

result = popular_news.run()

100%|██████████| 1309/1309 [01:36<00:00, 13.59it/s]


In [9]:
ptr = 0
for num in result:
    if ptr == 11:
        break
    
    score = 0
    print(num)
    print(len(result[num]))
    for data in result[num]:
        print(data[1], data[2])
    
    print("====="*10)

    ptr += 1

317
39
外資獲利了結賣超逾萬張 台積電早盤最低523元 https://money.udn.com//money/story/5607/6954791?from=edn_subcatelist_cate
紐約州共同退休基金減持台積電 兩大原因曝光 https://money.udn.com//money/story/5612/6954400?from=edn_subcatelist_cate
台積電跌10元 台股早盤跌逾百點 https://money.udn.com//money/story/5607/6952154?from=edn_subcatelist_cate
台積電股價開跌10元 早盤最低528元 https://money.udn.com//money/story/5607/6952330?from=edn_subcatelist_cate
台積電開盤下挫10元 台股跌逾百點暫失15500點 https://money.udn.com//money/story/5607/6952186?from=edn_subcatelist_cate
國研院半導體中心攜手台積電 導入7奈米晶片製作服務 https://money.udn.com//money/story/5612/6952767?from=edn_subcatelist_cate
國研院半導體中心導入台積電虛擬製程晶片設計訓練套件 https://money.udn.com//money/story/5612/6952670?from=edn_subcatelist_cate
接軌業界FinFET技術 半導體中心引入台積電訓練套件 https://money.udn.com//money/story/5612/6952662?from=edn_subcatelist_cate
台積電最後一盤爆7,194張成交量 收最低526元、市值滑落14兆 https://money.udn.com//money/story/5607/6953007?from=edn_subcatelist_cate
台積電重挫16元 台股回檔209.84點失守15400點 https://money.udn.com//money/story/5607/6952914?from=edn_subcatelist_cate
世